In [1]:
import geopandas as gpd
import os
from sqlalchemy import create_engine

In [2]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [3]:
sql = '''
SELECT 
bike_segid AS id,
name,
ftypedes AS type,
status,
source,
Shape.STAsBinary() AS geom
FROM dbo.BikeFacility;
'''

In [4]:
bikeways = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

In [5]:
bikeways.head()

,id,name,type,status,source,geom
0,NaN,Fox Hollow,Paved Shoulder,Built,LCOG,"LINESTRING (4235685.620 849995.967, 4235843.30..."
1,1.0,None,Paved Shoulder,Built,LCOG,"LINESTRING (4216611.495 857908.927, 4216569.84..."
2,2.0,Fox Hollow,Bike Route,Built,LCOG,"LINESTRING (4240496.355 857938.484, 4240518.05..."
3,3.0,Fox Hollow,Bike Route,Built,LCOG,"LINESTRING (4240082.967 859115.739, 4240067.37..."
4,4.0,None,Paved Shoulder,Built,LCOG,"LINESTRING (4237189.007 859678.425, 4237156.94..."


In [6]:
bikeways.columns

Index(['id', 'name', 'type', 'status', 'source', 'geom'], dtype='object')

In [7]:
bikeways['status'].unique()

array(['Built', 'Future', None], dtype=object)

In [8]:
bikeways['type'].unique()

array(['Paved Shoulder', 'Bike Route',
       'Pedestrian and Bicycle Access Way', 'Bike Lane',
       'Shared Use Path', 'Neighborhood Greenway', 'Sidewalk',
       'Connectivity for Routing', None], dtype=object)

In [9]:
bikeways[bikeways['status'] == 'Future']['type'].unique()

array(['Bike Lane', 'Shared Use Path',
       'Pedestrian and Bicycle Access Way', 'Bike Route'], dtype=object)

In [10]:
bikeways = bikeways[bikeways['status'] != 'Future']

In [11]:
outpath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\PerformanceAnalysis\sidewalk_bikeway_trails'

In [13]:
bikeways.crs = "EPSG:2914"

In [14]:
bikeways = bikeways.to_crs(epsg=3857)

In [15]:
bikeways.to_file(os.path.join(outpath, "bikeways.shp"))